In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import ipywidgets as widgets
from ipywidgets import (
    IntText,
    FloatText,
    RadioButtons,
    BoundedIntText,
    Checkbox,
    Layout,
    Output,
    Button,
    DatePicker,
    Dropdown,
    Label,
    HBox,
    HTML,
    VBox
)
import warnings

df_path = "./weight_loss_dfs/jordan_df.pqt"

In [ ]:
class ParquetCachedDF:
    def __init__(self, file_path):
        self.file_path = file_path
        if os.path.isfile(file_path):
            self.df = pd.read_parquet(file_path)
        else:
            self.df = None
    
    def get_df(self):
        return self.df
    
    def save_df(self):
        if self.df is None:
            raise ValueError("Cannot save empty dataframe")
        else:
            self.df.to_parquet(self.file_path)
            
    def set_df(self, df):
        self.df = df

In [ ]:
class DataEntryWidget:
    def __init__(self, label, layout=None):
        self.label = HTML(f"<b>{label}:</b>", layout=Layout(width="100px"))
        if layout is None:
            self.layout = Layout(width="max-content")
        else:
            self.layout = layout
        
    def to_box(self):
        raise NotImplementedError("must implement to_box")
    
class IntInput(DataEntryWidget):
    def __init__(self, label, value=0, disabled=False, layout=None):
        super(IntInput, self).__init__(label=label, layout=layout)
        
        self.input = IntText(
            value=value,
            disabled=disabled
        )
        self.value = value
        self.disabled = disabled
        
    def to_box(self):
        return HBox([self.label, self.input], layout=self.layout)
    
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        if not (value is None or np.isnan(value)):
            self.input.value = value
        else:
            self.input.value = 0
    
class FloatInput(DataEntryWidget):
    def __init__(self, label, value=0, disabled=False, layout=None):
        super(FloatInput, self).__init__(label=label, layout=layout)
        
        self.input = FloatText(
            value=value,
            disabled=disabled
        )
        self.value = value
        self.disabled = disabled
        
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        self.input.value = value
        
    def to_box(self):
        return HBox([self.label, self.input])
        
class TimeInput(DataEntryWidget):
    def __init__(self, label, value=0, disabled=False, layout=None):
        super(TimeInput, self).__init__(label=label, layout=layout)
        self.minutes_input = IntText(
            disabled=disabled
        )
        
        self.seconds_input = IntText(
            disabled=disabled
        )
        self.value = value
        self.disabled = disabled
        
    @property
    def disabled(self):
        return self._disabled
    
    @disabled.setter
    def disabled(self, value):
        self.minutes_input.disabled = value
        self.seconds_input.disabled = value
        self._disabled = value
    
    @property
    def value(self):
        return self.minutes_input.value + (self.seconds_input.value / 60)
    
    @value.setter
    def value(self, value):
        if isinstance(value, float) and not np.isnan(value):
            self.minutes_input.value = int(value)
            self.seconds_input.value = int((value - int(value)) * 60)
        else:
            self.minutes_input.value = 0
            self.seconds_input.value = 0
    
    def to_box(self):
        return HBox([self.label, self.minutes_input, HTML("m"), self.seconds_input, HTML("s")], layout=self.layout)
    
    
class InchesInput(DataEntryWidget):
    def __init__(self, label, disabled=False, layout=None, value=None):
        super(InchesInput, self).__init__(label=label, layout=layout)
        self.inches_input = IntText(
            step=1,
            disabled=disabled
        )
        
        self.numerator_input = IntText(
            step=1,
            disabled=disabled
        )
        
        self.denominator_input = Dropdown(
            options=[2, 4, 8, 16, 32, 64],
            disabled=disabled
        )
        self.value = value
        self.disabled = disabled
        
    def to_box(self):
        return HBox([self.label, self.inches_input, HTML("and"), self.numerator_input, HTML("/"), self.denominator_input])
    
    @property
    def value(self):
        return self.inches_input.value + (self.numerator_input.value / self.denominator_input.value)
    
    @value.setter
    def value(self, value):
        if value is not None and not np.isnan(value):
            value = float(value).as_integer_ratio()
            inches = value[0] // value[1]
            numerator = value[0] % value[1]
            denominator = value[1]
        else:
            inches = 0
            numerator = 0
            denominator = 8
        
        self.inches_input.value = inches
        self.numerator_input.value = numerator
        self.denominator_input.value = denominator if denominator != 1 else 8
    
    @property
    def disabled(self):
        return self.disabled
    
    @disabled.setter
    def disabled(self, value):
        self._disabled = value
        self.inches_input.disabled = value
        self.numerator_input.disabled = value
        self.denominator_input.disabled = value
        
class DateInput(DataEntryWidget):
    def __init__(self, label, disabled=False, layout=None, value=None):
        self.label = HTML(f"<b>{label}:</b>", layout=Layout(width="100px"))
        
        self.date_input = DatePicker(
            value=value,
            disabled=disabled
        )
        if value is None:
            self.value = datetime.date.today()
        else:
            self.value = value
            
        self.disabled = disabled
           
    def to_box(self):
        return HBox([self.label, self.date_input])
    
    @property
    def value(self):
        return self.date_input.value
    
    @value.setter
    def value(self, value):
        self.date_input.value = value
        
    @property
    def disabled(self):
        return self.date_input.disabled
    
    @disabled.setter
    def disabled(self, value):
        self.date_input.disabled = value
        
    def observe(self, func, names):
        self.date_input.observe(func, names=names)
        
class BoolInput(DataEntryWidget):
    def __init__(self, label, disabled=False, layout=None, value=None):
        super(BoolInput, self).__init__(label=label, layout=layout)
        self.input = Checkbox(disabled=disabled, indent=False)
        self.disabled = disabled
        self.value = value  
        
    @property
    def disabled(self):
        return self.input.disabled
    
    @disabled.setter
    def disabled(self, value):
        self.input.disabled = value
        
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        if isinstance(value, bool):
            self.input.value = value
        else:
            self.input.value = False
        
    def to_box(self):
        return HBox([self.label, self.input])
    
class SelectionInput(DataEntryWidget):
    def __init__(self, label, options, disabled=False, layout=None, value=None):
        super(SelectionInput, self).__init__(label=label, layout=layout)

        self.input = Dropdown(options=options)
        
    @property
    def disabled(self):
        return self.input.disabled
    
    @disabled.setter
    def disabled(self, value):
        self.input.disabled = value
        
    @property
    def value(self):
        return self.input.value
    
    @value.setter
    def value(self, value):
        self.input.value = value
        
    def to_box(self):
        return HBox([self.label, self.input])

In [ ]:
class DataEntry:
    def __init__(self, df_manager, date_entry_widget, data_entry_recipe, output_obj, form_layout=None):
        
        for item in data_entry_recipe:
            if not isinstance(item['widget'], DataEntryWidget):
                raise ValueError("Bad Not good!")
                
        self.df_manager = df_manager
        self.data_entry_recipe = data_entry_recipe
        self.date_entry_widget = date_entry_widget
        self.df = df_manager.df
        self.output_obj = output_obj
        
        if form_layout is None:
            self.form_layout = Layout()
        else:
            self.form_layout = form_layout
            
        self.submit_button = Button(
            description="Submit",
            disabled=False,
            button_style="success",
            tooltip="submit form",
            icon="check",
            layout=Layout(
                width="max-content"
            )
        )
        
        def handle_submit(e):
            self.add_row(e)
        self.submit_button.on_click(handle_submit)

        def handle_date_entry(e):
            self.set_defaults(date=e["new"])
        self.date_entry_widget.observe(handle_date_entry, names="value")
        
        form_children = [self.date_entry_widget.to_box()]
        form_children += [item['widget'].to_box() for item in self.data_entry_recipe]
        form_children += [self.submit_button]
        
        self.form = VBox(children=form_children, layout=self.form_layout)
        self.set_defaults()
        
        with self.output_obj:
            display(self.form)
            display(self.df_manager.get_df().head())
        
    def add_row(self, e):
        idx = str(self.date_entry_widget.value)
        if idx == "None":
            self.output_object.clear_output()
            display(self.form)
            display(HTML("Must enter a date!"))
        else:
            row = {item["col_title"]: item["widget"].value for item in self.data_entry_recipe}

            for key in row.keys():
                if not isinstance(row[key], bool):
                    if row[key] == 0:
                        row[key] = np.nan

            df = self.df_manager.get_df()

            if self.df is None:
                df = pd.DataFrame(
                    data=row,
                    index=[idx]
                )
                self.df_manager.set_df(df.sort_index(ascending=False))
                self.df_cache.save_df()
                with self.output_obj:
                    self.output_obj.clear_output()
                    display(form)
                    display(HTML("Success! New dataset created"))
                    display(df_cache.df)
            else:
                if idx in self.df.index:

                    def overwrite(e):
                        for k, v in row.items():
                            self.df.loc[idx, k] = v
                        self.df_manager.set_df(df.sort_index(ascending=False))
                        self.df_manager.save_df()
                        with self.output_obj:
                            self.output_obj.clear_output()
                            display(self.form)
                            display(HTML("Success! Row overwritten with new values"))
                            display(self.df_manager.get_df())

                    overwrite_btn = Button(
                        description="Data for date already exists! Overwrite row?",
                        disabled=False,
                        button_style="danger",
                        tooltip="overwrite row",
                        icon="check",
                        layout=Layout(
                            width="max-content"
                        )
                    )
                    overwrite_btn.on_click(overwrite)

                    with self.output_obj:
                        display(overwrite_btn)

                else:
                    self.df_manager.set_df(self.df.append(pd.Series(row, name=idx)).sort_index(ascending=False))
                    self.df_manager.save_df()

                    with self.output_obj:
                        self.output_obj.clear_output()
                        display(self.form)
                        display(HTML("Success!!"))
                        display(self.df_manager.get_df()) 

    def set_defaults(self, date=None):
        if date is None:
            date = datetime.date.today()
        if self.df is not None and str(date) in self.df.index:
            df_row = self.df.loc[str(date)]

            for item in self.data_entry_recipe:
                try:
                    item["widget"].value = item["type"](df_row[item["col_title"]])
                except:
                    warnings.warn(f"Could not get default value for {item['col_title']}")
                    item["widget"].value = item["type"](item["default_val"])

            # woefully, must do this string-date-datetime conversion to get dates to a format that can be subtracted
            datetime_selected = datetime.datetime(day=date.day, month=date.month, year=date.year)
            datetime_last_entry = datetime.datetime.strptime(self.df.iloc[0].name, "%Y-%m-%d")
            days_since_last_entry = (datetime_selected - datetime_last_entry).days
            if days_since_last_entry >= 1:
                default_target_cals = self.df.iloc[0]["Target Calories"]
        else:
            for item in self.data_entry_recipe:
                item["widget"].value = item["type"](item["default_val"])

In [ ]:
# widgets
date = DateInput(label="Date")
weight = FloatInput(label="Weight")
body_fat_percent = FloatInput(label="Body Fat %")
waist = InchesInput(label="Waist")
belly = InchesInput(label="Belly")
hips = InchesInput(label="Hips")
bicep = InchesInput(label="Bicep")
chest = InchesInput(label="Chest")
thigh = InchesInput(label="Thigh")
calf = InchesInput(label="Calf")
target_cals = IntInput(label="Target Calories")
calories = IntInput(label="Net Calories")
heart_rate = IntInput(label="Resting HR")
mile_time = TimeInput(label="Mile Time")
mode = SelectionInput(label="Mode", options=["Cutting", "Bulking", "Deload"])
workout = BoolInput(label="Workout")
cardio = BoolInput(label="Cardio")
meditate = BoolInput(label="Meditate")
yoga = BoolInput(label="Yoga")

items_list = [
    # name: (widget, value_type, default_value)
    {
        "widget": weight, 
        "type": float, 
        "default_val": 0,
        "col_title": "Weight",
    },
    {
        "widget": body_fat_percent, 
        "type": float, 
        "default_val": 0,
        'col_title': "Body Fat",
    },
    {
        "widget": waist, 
        "type": float, 
        "default_val": 0,
        "col_title": "Waist",
    },
    {
        "widget": belly, 
        "type": float, 
        "default_val": 0,
        "col_title": "Belly",
    },
    {
        "widget": hips, 
        "type": float, 
        "default_val": 0,
        "col_title": "Hips",
    },
    {
        "widget": bicep, 
        "type": float, 
        "default_val": 0,
        "col_title": "Bicep",
    },
    {
        "widget": chest, 
        "type": float, 
        "default_val": 0,
        "col_title": "Chest",
    },
    {
        "widget": thigh, 
        "type": float, 
        "default_val": 0,
        "col_title": "Thigh",
    },
    {
        "widget": calf, 
        "type": float, 
        "default_val": 0,
        "col_title": "Calf",
    },
    {
        "widget": target_cals, 
        "type": float, 
        "default_val": 0,
        "col_title": "Target Calories",
    },
    {
        "widget": calories, 
        "type": float, 
        "default_val": 0,
        "col_title": "Net Calories",
    },
    {
        "widget": heart_rate, 
        "type": float, 
        "default_val": 0,
        "col_title": "Resting Heart Rate",
    },
    {
        "widget": mile_time, 
        "type": float, 
        "default_val": 0,
        "col_title": "Mile Time",
    },
    {
        "widget": cardio, 
        "type": bool, 
        "default_val": False,
        "col_title": "Cardio",
    },
    {
        "widget": workout, 
        "type": bool, 
        "default_val": False,
        "col_title": "Workout",
    },
    {
        "widget": meditate, 
        "type": bool, 
        "default_val": False,
        "col_title": "Meditate",
    },
    {
        "widget": yoga, 
        "type": bool, 
        "default_val": False,
        "col_title": "Yoga",
    },
    {
        "widget": mode, 
        "type": str, 
        "default_val": "Cutting",
        "col_title": "Mode",
    }
]

# initial setup
df_manager = ParquetCachedDF(file_path=df_path)
out = Output()

display(out)

engine = DataEntry(
    df_manager=df_manager,
    date_entry_widget=date,
    data_entry_recipe=items_list,
    output_obj=out,
)
